In [1]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
from torch import cuda
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

from datasets import load_dataset, concatenate_datasets

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'

print(device)

cuda


In [4]:
model_path = 'distilbert-base-uncased'

tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path, id2label={0:"NEG", 1:"POS"}, label2id={"NEG":0, "POS":1})



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
!rm -rf ~/.cache/huggingface/datasets


In [6]:
imdb_train = load_dataset('stanfordnlp/imdb', split='train' )
imdb_test = load_dataset('stanfordnlp/imdb', split='test[:6250]+test[-6250:]')
imdb_eval = load_dataset('stanfordnlp/imdb', split='test[6250:12500]+test[-12500:-6250]')


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
imdb_train.shape, imdb_test.shape, imdb_eval.shape

((25000, 2), (12500, 2), (12500, 2))

In [8]:
imdb_train= load_dataset('imdb', split="train[:2000]+train[-2000:]")
imdb_test= load_dataset('imdb',split="test[:500]+test[-500:]")
imdb_eval= load_dataset('imdb', split="test[500:1000]+test[-1000:-500]")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
train = imdb_train.map(lambda e: tokenizer(e['text'], padding=True, truncation=True), batched=True, batch_size=1000)
test = imdb_test.map(lambda e: tokenizer(e['text'], padding=True, truncation=True), batched=True, batch_size=1000)
eval= imdb_eval.map(lambda e: tokenizer(e['text'], padding=True,truncation=True), batched=True, batch_size=1000)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
import pandas as pd

pd.DataFrame(train)

,text,label,input_ids,attention_mask
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,"[101, 1045, 12524, 1045, 2572, 8025, 1011, 375...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"""I Am Curious: Yellow"" is a risible and preten...",0,"[101, 1000, 1045, 2572, 8025, 1024, 3756, 1000...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,If only to avoid making this type of film in t...,0,"[101, 2065, 2069, 2000, 4468, 2437, 2023, 2828...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,This film was probably inspired by Godard's Ma...,0,"[101, 2023, 2143, 2001, 2763, 4427, 2011, 2643...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"Oh, brother...after hearing about this ridicul...",0,"[101, 2821, 1010, 2567, 1012, 1012, 1012, 2044...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
3995,A hit at the time but now better categorised a...,1,"[101, 1037, 2718, 2012, 1996, 2051, 2021, 2085...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3996,I love this movie like no other. Another time ...,1,"[101, 1045, 2293, 2023, 3185, 2066, 2053, 2060...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3997,This film and it's sequel Barry Mckenzie holds...,1,"[101, 2023, 2143, 1998, 2009, 1005, 1055, 8297...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3998,'The Adventures Of Barry McKenzie' started lif...,1,"[101, 1005, 1996, 7357, 1997, 6287, 18506, 100...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [11]:
from transformers import TrainingArguments, Trainer

In [21]:
training_args= TrainingArguments(
      output_dir='./MYIMDBModel2', #points to the point checkpoints, and where the predictions will be saved.
      do_train=True,
      do_eval=True, #to monitor the model performance during the training and e
      report_to='none',
      num_train_epochs= 3, #transoformer-based fine-tunning satisfied with only this, due to transfer learning.
      per_device_train_batch_size=32,
      per_device_eval_batch_size=64,
      warmup_steps= 100, #Here, for the first 100 steps, the learning rate will linearly increase from 0 to the maximum value, then it will start decreasing
      weight_decay= 0.01, # specify the weight decay coefficient for regularization during training.
      logging_strategy='steps', # set log results after specified number of steps
      logging_dir='./logs2',
      logging_steps=100, #frequency of logging during training
      eval_strategy='steps',
      fp16= cuda.is_available(),
      load_best_model_at_end=True #Load the best model at the end and specify that the best model according to the evaluation metric should be loaded at the end of training.
)

**Evaluation Metric**
allows us to monitor the progress of the training in terms of metrics such as precision, recall, RMSE, Pearson Correlation

In [13]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metric(pred):
  labels= pred.label_ids
  preds= pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
  acc = accuracy_score(labels, preds)

  return {
      'Accuracy': acc,
      'F1': f1,
      'Precision': precision,
      'Recall': recall
  }


In [22]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset= train,
    eval_dataset= eval,
    compute_metrics= compute_metric
)

results = trainer.train() #choose the least evaluation loss model 'eval_loss'

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.507400,0.352690,0.857000,0.855467,0.872821,0.857000
200,0.237800,0.277813,0.901000,0.900983,0.901271,0.901000
300,0.152000,0.317338,0.903000,0.902957,0.903712,0.903000


**Evaluate Checkpoints**

In [23]:
q=[trainer.evaluate(eval_dataset=data) for data in [train, test , eval]]

pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]

,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.048343,0.9885,0.988500,0.988504,0.9885
val,0.285932,0.9150,0.914955,0.915880,0.9150
test,0.341867,0.9020,0.901994,0.902103,0.9020


In [24]:
def get_prediction(text):
    inputs = tokenizer(text, padding=True,truncation=True, max_length=250, return_tensors="pt").to(device)
    outputs = model(inputs["input_ids"].to(device), inputs["attention_mask"].to(device))
    probs = outputs[0].softmax(1)
    return probs, probs.argmax()

In [43]:
text = "I didn't like the movie it bored me "
get_prediction(text)[1].item()

0

**Save the best model for further inference:**



In [44]:
model_save_path = "MyBestIMDBModel"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

('MyBestIMDBModel/tokenizer_config.json',
 'MyBestIMDBModel/special_tokens_map.json',
 'MyBestIMDBModel/vocab.txt',
 'MyBestIMDBModel/added_tokens.json',
 'MyBestIMDBModel/tokenizer.json')

In [51]:
from transformers import pipeline , DistilBertForSequenceClassification, DistilBertTokenizerFast

model = DistilBertForSequenceClassification.from_pretrained("MyBestIMDBModel")

tokenizer= DistilBertTokenizerFast.from_pretrained("MyBestIMDBModel")
SA_model= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
SA_model("the movie was very impressive")

Device set to use cuda:0


[{'label': 'POS', 'score': 0.9792680740356445}]